In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def linear_kernel(x, y, b=1):
    """Returns the linear combination of arrays `x` and `y` with
    the optional bias term `b` (set to 1 by default)."""
    
    return np.dot(x,y.T) + b

In [3]:
# Objective function to optimize (OR OPTIMIZER)

def objective_function(alphas, target, kernel, X_train):
    """Returns the SVM objective function based in the input model defined by:
    `alphas`: vector of Lagrange multipliers
    `target`: vector of class labels (-1 or 1) for training data
    `kernel`: kernel function
    `X_train`: training data for model."""
    
    return np.sum(alphas) - 0.5 * np.sum(target * target * kernel(X_train, X_train) * alphas * alphas)



In [4]:
def take_step(i1, i2, model):
    
    # Skip if chosen alphas are the same
    if i1 == i2:
        return 0, model
    
    alph1 = model.alphas[i1]
    alph2 = model.alphas[i2]
    y1 = model.y[i1]
    y2 = model.y[i2]
    E1 = model.errors[i1]
    E2 = model.errors[i2]
    s = y1 * y2
    
    # Compute L & H, the bounds on new possible alpha values
    if (y1 != y2):
        L = max(0, alph2 - alph1)
        H = min(model.C, model.C + alph2 - alph1)
    elif (y1 == y2):
        L = max(0, alph1 + alph2 - model.C)
        H = min(model.C, alph1 + alph2)
    if (L == H):
        return 0, model

    # Compute kernel & 2nd derivative eta
    k11 = model.kernel(model.X[i1], model.X[i1])
    k12 = model.kernel(model.X[i1], model.X[i2])
    k22 = model.kernel(model.X[i2], model.X[i2])
    eta = 2 * k12 - k11 - k22
    
    # Compute new alpha 2 (a2) if eta is negative
    if (eta < 0):
        a2 = alph2 - y2 * (E1 - E2) / eta
        # Clip a2 based on bounds L & H
        if L < a2 < H:
            a2 = a2
        elif (a2 <= L):
            a2 = L
        elif (a2 >= H):
            a2 = H
            
    # If eta is non-negative, move new a2 to bound with greater objective function value
    else:
        alphas_adj = model.alphas.copy()
        alphas_adj[i2] = L
        # objective function output with a2 = L
        Lobj = objective_function(alphas_adj, model.y, model.kernel, model.X) 
        alphas_adj[i2] = H
        # objective function output with a2 = H
        Hobj = objective_function(alphas_adj, model.y, model.kernel, model.X)
        if Lobj > (Hobj + eps):
            a2 = L
        elif Lobj < (Hobj - eps):
            a2 = H
        else:
            a2 = alph2
            
    # Push a2 to 0 or C if very close
    if a2 < 1e-8:
        a2 = 0.0
    elif a2 > (model.C - 1e-8):
        a2 = model.C
    
    # If examples can't be optimized within epsilon (eps), skip this pair
    if (np.abs(a2 - alph2) < eps * (a2 + alph2 + eps)):
        return 0, model
    
    # Calculate new alpha 1 (a1)
    a1 = alph1 + s * (alph2 - a2)
    
    # Update threshold b to reflect newly calculated alphas
    # Calculate both possible thresholds
    b1 = E1 + y1 * (a1 - alph1) * k11 + y2 * (a2 - alph2) * k12 + model.b
    b2 = E2 + y1 * (a1 - alph1) * k12 + y2 * (a2 - alph2) * k22 + model.b
    
    # Set new threshold based on if a1 or a2 is bound by L and/or H
    if 0 < a1 and a1 < C:
        b_new = b1
    elif 0 < a2 and a2 < C:
        b_new = b2
    # Average thresholds if both are bound
    else:
        b_new = (b1 + b2) * 0.5

    # Update model object with new alphas & threshold
    model.alphas[i1] = a1
    model.alphas[i2] = a2
    
    # Update error cache
    # Error cache for optimized alphas is set to 0 if they're unbound
    for index, alph in zip([i1, i2], [a1, a2]):
        if 0.0 < alph < model.C:
            model.errors[index] = 0.0
    
    # Set non-optimized errors based on equation 12.11 in Platt's book
    non_opt = [n for n in range(model.m) if (n != i1 and n != i2)]
    model.errors[non_opt] = model.errors[non_opt] +\
                            y1*(a1 - alph1)*model.kernel(model.X[i1], model.X[non_opt]) +\
                            y2*(a2 - alph2)*model.kernel(model.X[i2], model.X[non_opt]) + model.b - b_new
    
    # Update model threshold
    model.b = b_new
    
    return 1, model

In [5]:
def examine_example(i2, model):
    
    y2 = model.y[i2]
    alph2 = model.alphas[i2]
    E2 = model.errors[i2]
    r2 = E2 * y2

    # Proceed if error is within specified tolerance (tol)
    if ((r2 < -tol and alph2 < model.C) or (r2 > tol and alph2 > 0)):
        
        if len(model.alphas[(model.alphas != 0) & (model.alphas != model.C)]) > 1:
            # Use 2nd choice heuristic is choose max difference in error
            if model.errors[i2] > 0:
                i1 = np.argmin(model.errors)
            elif model.errors[i2] <= 0:
                i1 = np.argmax(model.errors)
            step_result, model = take_step(i1, i2, model)
            if step_result:
                return 1, model
            
        # Loop through non-zero and non-C alphas, starting at a random point
        for i1 in np.roll(np.where((model.alphas != 0) & (model.alphas != model.C))[0],
                          np.random.choice(np.arange(model.m))):
            step_result, model = take_step(i1, i2, model)
            if step_result:
                return 1, model
        
        # loop through all alphas, starting at a random point
        for i1 in np.roll(np.arange(model.m), np.random.choice(np.arange(model.m))):
            step_result, model = take_step(i1, i2, model)
            if step_result:
                return 1, model
    
    return 0, model

In [6]:
class SMOModel:
    """Container object for the model used for sequential minimal optimization."""
    
    def __init__(self, X, y, C, kernel, alphas, b, errors):
        self.X = X               # training data vector
        self.y = y               # class label vector
        self.C = C               # regularization parameter
        self.kernel = kernel     # kernel function
        self.alphas = alphas     # lagrange multiplier vector
        self.b = b               # scalar bias term
        self.errors = errors     # error cache
        self._obj = []           # record of objective function value
        self.m = len(self.X)     # store size of training set

In [84]:
def train(model):
    
    numChanged = 0
    examineAll = 1
    iterations = 0
    
    while(numChanged > 0) or (examineAll):
        iterations += 1
#         print(iterations)
        if(iterations>5000):
            break
        numChanged = 0
        if examineAll:
            # loop over all training examples
            for i in range(model.alphas.shape[0]):
                examine_result, model = examine_example(i, model)
                numChanged += examine_result
                if examine_result:
                    obj_result = objective_function(model.alphas, model.y, model.kernel, model.X)
                    model._obj.append(obj_result)
        else:
            # loop over examples where alphas are not already at their limits
            for i in np.where((model.alphas != 0) & (model.alphas != model.C))[0]:
                examine_result, model = examine_example(i, model)
                numChanged += examine_result
                if examine_result:
                    obj_result = objective_function(model.alphas, model.y, model.kernel, model.X)
                    model._obj.append(obj_result)
        if examineAll == 1:
            examineAll = 0
        elif numChanged == 0:
            examineAll = 1
        
    return model

In [85]:
# Function that gives predictions
def predict_decision(alphas, target, kernel, X_train, x_test, b):
    """Applies the SVM decision function to the input feature vectors in `x_test`."""
    
    result = np.dot((alphas*target),kernel(X_train, x_test))-b
    return result

## Using SVM For Binary(2 class) Classification

In [70]:
# Set model parameters and initial values
C = 1.0
#m = len(X_train)
#initial_alphas = np.zeros(m)
initial_b = 0.0

# Set tolerances
tol = 0.01 # error tolerance
eps = 0.01 # alpha tolerance

# Instantiate model
#model = SMOModel(X_train, y_train, C, linear_kernel,initial_alphas, initial_b, np.zeros(m))
#output = train(model)

# For prediction
#predictions = decision_function(model.alphas, model.y, model.kernel,model.X, X_test, model.b)
#print(predictions)

## For Multiclass Classification Using SVM

In [10]:
import pandas as pd

In [11]:
df = pd.read_csv('winequality-white.csv',delimiter=';')

In [14]:
X=df.iloc[:,df.columns!='quality'].values
Y=df.iloc[:,df.columns=='quality'].values.flatten()

In [15]:
from sklearn import preprocessing
#normalizing
normalized_X= preprocessing.normalize(X)
standardized_X = preprocessing.scale(X)

In [16]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(standardized_X,Y,test_size=0.2,)

In [17]:
x_train.shape

(3918, 11)

## Trying Out One Vs Rest (OVR)

##  TRAINING PART

In [71]:
np.unique(y_train)  ## purai array ma unique values haru matrai select garne 

array([3, 4, 5, 6, 7, 8, 9])

In [72]:
# Set model parameters and initial values
C = 1.0
m = len(x_train)
initial_alphas = np.zeros(m)
initial_b = 0.0

In [73]:
import copy

In [86]:
labels = np.unique(y_train)
classifiers=[]
# iterate over all labels ..
for label in labels:
    print ("Classifier",label," vs Rest")
    yi = np.array(y_train)
    yi[yi != label] = -1.0
    yi[yi == label] = 1.0
    print(yi)
    model = SMOModel(x_train, y_train, C, linear_kernel,initial_alphas, initial_b, np.zeros(m))
    train(model)
    classifiers.append(copy.deepcopy(model))

Classifier 3  vs Rest
[-1 -1 -1 ... -1 -1 -1]
Classifier 4  vs Rest
[-1 -1 -1 ... -1 -1 -1]
Classifier 5  vs Rest
[-1  1 -1 ...  1 -1  1]
Classifier 6  vs Rest
[-1 -1  1 ... -1  1 -1]
Classifier 7  vs Rest
[ 1 -1 -1 ... -1 -1 -1]
Classifier 8  vs Rest
[-1 -1 -1 ... -1 -1 -1]
Classifier 9  vs Rest
[-1 -1 -1 ... -1 -1 -1]


In [87]:
print("Total no of classifier created ",len(classifiers))

Total no of classifier created  7


## PREDICTING PART

In [88]:
labels

array([3, 4, 5, 6, 7, 8, 9])

In [89]:
# we have to return prediction for x_test array of 11 features Dimension[x,11]
n = x_test.shape[0]
scores = np.zeros((n, len(labels)))
scores.shape #each row of scores will store score(probability) for each label(class) and we will return largest one

(980, 7)

In [92]:
for i in range(len(labels)):
    model = classifiers[i]
    scores[:,i] = predict_decision(model.alphas, model.y, model.kernel,model.X, x_test, model.b)

In [93]:
#np.argmax(scores, axis=1)
scores

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])